# AI Launch Lab - Sea Ice Movement Challenge - PCA and Training

Do some more analysis on the data before choosing our final model.

In [1]:
%matplotlib inline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
import pandas as pd
import pickle
import os 
pd.options.mode.chained_assignment = None
import datetime
import numpy as np
import os
import zipfile
import modules.ml_pipeline.readdata as mlpp

Load the data from disk and display it

In [2]:
# unzip the zip dataset
with zipfile.ZipFile('data/converted.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

# Load the raw data to disk
input_path = "data/converted.csv"
df = pd.read_csv(input_path)

# Convert all column names to lower case and display the dataframe 
df = df.rename(str.lower, axis='columns')
df.set_index("id_buoy")

# Print the dataframe dimensions
print("Dataframe shape: ", df.shape)
display(df)

# Print dataframe statistics
display(df.describe())

Dataframe shape:  (339478, 15)


,year,month,day,doy,id_buoy,sic_cdr,h_cs2smos,h_piomas,d2c,buoy_lat,buoy_lon,buoy_vel_mag,buoy_vel_dir,wind_vel_mag,wind_vel_dir
0,1979,2,18,49,1906,0.990195,NaN,3.189743,522.523298,78.007070,-128.549129,1.370671,2.191824,6.711849,3.189490
1,1979,2,18,49,1913,0.966372,NaN,2.484009,412.767669,74.498024,-119.750294,0.741408,0.520564,6.851881,3.240164
2,1979,2,18,49,1914,0.996022,NaN,2.474106,362.547379,74.003619,-134.786524,1.187695,2.934923,8.896751,3.014921
3,1979,2,18,49,1918,0.982681,NaN,3.740522,381.025629,81.019593,-145.578020,0.920127,0.028026,1.496117,3.905953
4,1979,2,19,50,1906,0.990302,NaN,3.188522,521.535334,78.002077,-128.560665,1.300527,1.273525,3.338513,2.278041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339473,2019,12,30,364,44880,1.000000,1.414148,1.620020,702.312813,86.513431,-29.153877,11.739478,4.628723,6.522125,4.314615
339474,2019,12,30,364,53005,1.000000,1.039972,1.288953,360.491321,79.286760,-53.579091,8.973035,3.772087,4.630856,3.224141
339475,2019,12,30,364,95020,1.000000,2.059716,1.960349,393.799208,79.025667,-135.924079,2.365742,1.967356,2.240471,0.523007
339476,2019,12,30,364,7750,1.000000,1.411272,1.610893,680.057567,86.582037,-23.247546,11.792136,4.703807,5.904344,4.281085


,year,month,day,doy,id_buoy,sic_cdr,h_cs2smos,h_piomas,d2c,buoy_lat,buoy_lon,buoy_vel_mag,buoy_vel_dir,wind_vel_mag,wind_vel_dir
count,339478.000000,339478.000000,339478.000000,339478.000000,339478.000000,339478.000000,72768.000000,339478.000000,339478.000000,339478.000000,339478.000000,339478.000000,339478.000000,339478.000000,339478.000000
mean,2007.013774,6.749374,15.744955,190.020314,35576.107397,0.955278,1.774036,2.069873,418.132032,79.459657,-60.113573,8.353460,2.811403,5.356477,3.070813
std,9.822726,3.454148,8.752622,105.562830,31444.526635,0.124625,0.800483,0.922070,259.827915,4.921463,100.397647,7.690366,1.777374,2.665124,1.790118
min,1979.000000,1.000000,1.000000,1.000000,0.000000,0.000027,0.016601,0.000000,0.359009,50.547465,-178.357062,0.000000,-0.000000,0.010611,0.000018
25%,2000.000000,4.000000,8.000000,101.000000,7408.000000,0.976706,1.215125,1.478507,210.220330,75.589094,-130.750681,2.888761,1.334609,3.411475,1.560836
50%,2009.000000,7.000000,16.000000,190.000000,26150.000000,1.000000,1.733047,2.041238,389.570385,80.007813,-107.140144,6.664808,2.603533,5.076378,2.990427
75%,2015.000000,10.000000,23.000000,285.000000,64878.000000,1.000000,2.284967,2.619901,599.120875,83.595352,-0.258598,11.674104,4.201071,6.950391,4.580908
max,2019.000000,12.000000,31.000000,364.000000,99800.000000,1.000000,4.930797,6.284669,1120.364872,88.734181,179.999301,75.609704,6.283168,25.356925,6.283139


In [3]:
# Do some manipulations on the data and clean it (remove rows of all NaNs, remove duplicates, etc.)

# Dropping the NA when there is no value on the current columns
print("Before dropping the NaNs :", df.shape)

# Remove any rows that have buoy velocity/mag =0 
df = df.drop(df[df["buoy_vel_mag"] == 0].index)
df = df.reset_index()

# create a placeholder for ice thickness
temp = [None]*df.shape[0]
print(len(temp))

Before dropping the NaNs : (339478, 15)
330371


In [4]:
# iterate through all the rows and check if thickness is 0
for index, row in df.iterrows():
    #print("index", index)
    if row["h_piomas"] == 0 :
        
        if row["h_cs2smos"] == None: 
            pass
        else: 
            temp[index] = row["h_cs2smos"]
    else:
        if row["h_cs2smos"] == None:
            temp[index] = row["h_piomas"]
        else: 
            temp[index] = (row["h_cs2smos"] + row["h_piomas"])/2

df["ice_thickness"] = temp            
display(df["ice_thickness"])

0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
            ...   
330366    0.933125
330367    1.517084
330368    1.164462
330369    2.010032
330370    1.511082
Name: ice_thickness, Length: 330371, dtype: float64

In [5]:
# Print size of the df after dropping duplicates
#print("After dropping the duplicates:", df.shape)

# Drop any rows where ice_thickness is NaN
df = df.dropna(subset=["h_piomas"])

display(df)

,index,year,month,day,doy,id_buoy,sic_cdr,h_cs2smos,h_piomas,d2c,buoy_lat,buoy_lon,buoy_vel_mag,buoy_vel_dir,wind_vel_mag,wind_vel_dir,ice_thickness
0,0,1979,2,18,49,1906,0.990195,NaN,3.189743,522.523298,78.007070,-128.549129,1.370671,2.191824,6.711849,3.189490,NaN
1,1,1979,2,18,49,1913,0.966372,NaN,2.484009,412.767669,74.498024,-119.750294,0.741408,0.520564,6.851881,3.240164,NaN
2,2,1979,2,18,49,1914,0.996022,NaN,2.474106,362.547379,74.003619,-134.786524,1.187695,2.934923,8.896751,3.014921,NaN
3,3,1979,2,18,49,1918,0.982681,NaN,3.740522,381.025629,81.019593,-145.578020,0.920127,0.028026,1.496117,3.905953,NaN
4,4,1979,2,19,50,1906,0.990302,NaN,3.188522,521.535334,78.002077,-128.560665,1.300527,1.273525,3.338513,2.278041,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330366,339472,2019,12,30,364,25560,1.000000,0.805200,1.061049,470.197315,75.686986,-96.341501,17.168005,3.811734,8.234026,3.676847,0.933125
330367,339473,2019,12,30,364,44880,1.000000,1.414148,1.620020,702.312813,86.513431,-29.153877,11.739478,4.628723,6.522125,4.314615,1.517084
330368,339474,2019,12,30,364,53005,1.000000,1.039972,1.288953,360.491321,79.286760,-53.579091,8.973035,3.772087,4.630856,3.224141,1.164462
330369,339475,2019,12,30,364,95020,1.000000,2.059716,1.960349,393.799208,79.025667,-135.924079,2.365742,1.967356,2.240471,0.523007,2.010032


In [35]:
print("Sea Ice Movement Datasets")

training_targets = df[["buoy_vel_mag","buoy_vel_dir"]]

# We will leave the month for now because it could be an indicator of weather/season 
print("Target Variables")
display(training_targets)

Sea Ice Movement Datasets
Target Variables


,buoy_vel_mag,buoy_vel_dir
0,1.370671,2.191824
1,0.741408,0.520564
2,1.187695,2.934923
3,0.920127,0.028026
4,1.300527,1.273525
...,...,...
330366,17.168005,3.811734
330367,11.739478,4.628723
330368,8.973035,3.772087
330369,2.365742,1.967356


In [36]:
# Drop the time related columns 
training_data = df.drop(["index", "year","day","h_cs2smos", "ice_thickness", "sic_cdr", "doy", "id_buoy","buoy_vel_mag","buoy_vel_dir"], axis = 1)
display(training_data)

,month,h_piomas,d2c,buoy_lat,buoy_lon,wind_vel_mag,wind_vel_dir
0,2,3.189743,522.523298,78.007070,-128.549129,6.711849,3.189490
1,2,2.484009,412.767669,74.498024,-119.750294,6.851881,3.240164
2,2,2.474106,362.547379,74.003619,-134.786524,8.896751,3.014921
3,2,3.740522,381.025629,81.019593,-145.578020,1.496117,3.905953
4,2,3.188522,521.535334,78.002077,-128.560665,3.338513,2.278041
...,...,...,...,...,...,...,...
330366,12,1.061049,470.197315,75.686986,-96.341501,8.234026,3.676847
330367,12,1.620020,702.312813,86.513431,-29.153877,6.522125,4.314615
330368,12,1.288953,360.491321,79.286760,-53.579091,4.630856,3.224141
330369,12,1.960349,393.799208,79.025667,-135.924079,2.240471,0.523007


Split the data into a Training and Test set 

In [ ]:
# Evaluate multioutput regression model 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import cross_val_score

Split the dataset into training and validation sets. 

In [40]:
# test_size: what proportion of original data is used for test set
train_data, test_data, train_labels, test_labels = train_test_split(
    training_data, training_targets, test_size= 0.25, shuffle=True)

# show the sizes of the training and test sets
print("Training data shape: ", train_data.shape)
print("Test data shape: ", test_data.shape)

print("Training labels shape: ", train_labels.shape)
print("Test labels shape: ", test_labels.shape)

Training data shape:  (247778, 7)
Test data shape:  (82593, 7)
Training labels shape:  (247778, 2)
Test labels shape:  (82593, 2)


In [41]:
max_depth = 30
regr_multirf = MultiOutputRegressor(RandomForestRegressor(n_estimators=100,
                                                          max_depth=max_depth,
                                                          random_state=0))
regr_multirf.fit(train_data, train_labels)

regr_rf = RandomForestRegressor(n_estimators=100, max_depth=max_depth,
                                random_state=2)
regr_rf.fit(train_data, train_labels)

RandomForestRegressor(max_depth=30, random_state=2)

In [42]:
# Predict on new data
y_multirf = regr_multirf.predict(test_data)
y_rf = regr_rf.predict(test_data)

In [43]:
print(regr_rf.score(test_data,test_labels))
print(regr_multirf.score(test_data,test_labels))

0.6137082003405874
0.6156692422774437


## PCA to Speed up Machine Learning Algorithms
(Taken from https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60)

Since PCA yields a feature subspace that maximizes the variance along the axes, it makes sense to standardize the data, especially, if it was measured on different scales.

Standardization of a dataset is a common requirement for many machine learning estimators: they might behave badly if the individual feature do not more or less look like standard normally distributed data

Normalize the data: 

In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

scaler = StandardScaler()

# Fit on training set only.
scaler.fit(train_data)

# Apply transform to both the training set and the test set.
train_data = scaler.transform(train_data)
test_data = scaler.transform(test_data)

# display the normalized data
display(train_data)

array([[ 1.5132759 , -0.20790203, -0.1405951 , ...,  0.33270902,
         1.58641061, -0.85393221],
       [-1.0966986 ,  0.19949706, -1.47611862, ..., -0.60376619,
        -1.3509636 ,  0.47555508],
       [-1.67669293, -1.21401387, -1.07012586, ...,  1.61687624,
         0.32970656, -0.50297661],
       ...,
       [ 1.5132759 , -0.67993322,  1.08119054, ...,  0.22190044,
        -0.43174002, -0.85694769],
       [-1.67669293,  0.53696031,  0.26074542, ...,  1.49002838,
         1.92398325, -0.89051873],
       [-0.2267071 ,  0.89789064,  1.18559124, ..., -0.61477872,
         1.01831252, -1.42284901]])

In [16]:
# Choose the minimum percentage of the variables such that 95% of the variance in the dataset is retained
pca = PCA(n_components=4) 

Fit PCA on training set. Note: you are fitting PCA on the training set only

In [17]:
pca.fit(train_data)

# print the number of components we are left with
print("Number of components: ", pca.n_components_)
print("PCA variance ratio: ", pca.explained_variance_ratio_)

# Transform the data according to the number of principal components
train_data = pca.transform(train_data)
test_data = pca.transform(test_data)

Number of components:  4
PCA variance ratio:  [0.22114785 0.19318859 0.18161576 0.14305846]


In [18]:
# Recreate the principal component dataframe: 
principalDf = pd.DataFrame(data = train_data, columns = ['principal component 1', 
                                                         'principal component 2','principal component 3', 'principal component 4'])

# Concatenate the new dataset with the targets
finalDf = pd.concat([principalDf, training_targets], axis = 1)

In [19]:

# check scikit-learn version
import sklearn
print(sklearn.__version__)

0.23.2


Create a series of models and calculate the predictions

Different model choices we can use include Linear Regression, K-Nearest Neighbors, Decision Trees

In [22]:
# Linear Regression

from sklearn.linear_model import LinearRegression

# define model
model = LinearRegression()
# fit model
model.fit(train_data, train_labels)

LinearRegression()

In [ ]:
# K-Nearest Neighbors

from sklearn.neighbors import KNeighborsRegressor

# define model
model = KNeighborsRegressor()
# fit model
model.fit(train_data, train_labels)

In [ ]:
# Decision Tree Regressor
from sklearn.tree import DecisionTreeRegressor

# define model
model = DecisionTreeRegressor()
# fit model
model.fit(train_data, train_labels)

In [25]:
# Evaluate the model with K-Cross Validation

IMPORT K-FOLD
# define the evaluation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3)

# evaluate the model and collect the scores
n_scores = cross_val_score(model, train_data, train_labels,  scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

# force the scores to be positive
n_scores = absolute(n_scores)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

#model.predict(test_data, test_labels)


MAE: 2.619 (0.011)


In [31]:
print(train_data.shape)

print(train_data)

print(train_labels.shape)

print(train_labels)


(247778, 4)
[[-0.35629487 -1.03165445  1.38859754  1.3815066 ]
 [ 1.49092881  2.17738976 -0.66566533 -1.15485404]
 [ 0.46169316  1.84582477  1.28159171  0.61132311]
 ...
 [-0.89370066 -2.19362746  0.79531656 -0.50915645]
 [-2.43512571  0.55062074  0.28954002  2.1691723 ]
 [-0.87765537 -0.71588742 -1.02775752  1.44189405]]
(247778, 2)
        buoy_vel_mag  buoy_vel_dir
24216      12.883657      1.578747
220702      0.004272      5.764045
141101     31.026777      2.068567
28895      19.987143      6.190919
166781     21.687009      1.912180
...              ...           ...
322894     12.209663      1.925925
47560      18.712242      1.894609
327618     13.200331      2.143159
276167     18.320172      1.768791
83770      14.015064      1.002127

[247778 rows x 2 columns]


In [30]:
print(test_data.shape)

print(test_data)

(82593, 4)
[[ 0.29397432  0.38829931 -0.40369334 -0.57500646]
 [ 1.40557112  0.21018173  0.31733122  1.77587227]
 [ 0.73890152  0.95838763 -0.73587296 -0.83505713]
 ...
 [ 1.26786955  1.7538985  -0.87345472  1.01206188]
 [-0.13544566 -0.0599247  -0.85347065  0.70959229]
 [ 1.3182832  -1.10651102  0.35559661  0.09663111]]


In [28]:
print(test_labels.shape)

(82593, 2)


In [29]:
print(test_labels)

        buoy_vel_mag  buoy_vel_dir
201870     16.030408      1.270452
282948     28.756294      3.542535
276694      2.830781      1.734293
234024      1.622393      0.186016
101406      7.473834      3.123019
...              ...           ...
53647       5.465394      4.971474
49394       1.754999      3.704154
288583      9.556922      5.514071
245787     11.456255      0.498476
284375     21.183223      2.494424

[82593 rows x 2 columns]
